# Use OpenRouter With OpenAI Agents SDK

Setup Prerequisite:

1. [Signup at OpenRouter](https://openrouter.ai/)
2. [Create an API Key](https://openrouter.ai/settings/keys)
2. Select a Free Model (you can continue as we are using a free model here)

## Free and Paid Models

The OpenRouter supports the latest DeepSeek V3 0324 and 50+ other models for free. Most of them support the defacto standard: OpenAI Chat Completion API.


If you are using a free model variant (with an ID ending in :free), then you will be limited to 20 requests per minute and 200 requests per day.

**See all Models List: https://openrouter.ai/models**

Note: OpenRouter do not charge anything extra at inference time.

## Rate Limiting and Crediting

There are a few rate limits that apply to certain types of requests, regardless of account status:

- Free limit: If you are using a free model variant (with an ID ending in :free), then you will be limited to 20 requests per minute and 200 requests per day.

If your account has a negative credit balance, you may see 402 errors, including for free models. Adding credits to put your balance above zero allows you to use those models again.

[Reference](https://openrouter.ai/docs/api-reference/limits)

## Install OpenAI Agents Dep.

In [1]:
!pip install -Uq openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.3/108.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.2/129.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.3 MB/s eta 0:00:00


In [2]:
import nest_asyncio
nest_asyncio.apply()

## Provider Config

In [10]:
from google.colab import userdata

OPENROUTER_API_KEY = userdata.get("OPENROUTER_API_KEY")

In [14]:
#Reference: https://openrouter.ai/docs/quickstart

BASE_URL = "https://openrouter.ai/api/v1"
MODEL = "google/gemma-3-1b-it:free"

# Some other free models on 26th March:
# https://openrouter.ai/deepseek/deepseek-chat-v3-0324:free
# https://openrouter.ai/google/gemini-2.5-pro-exp-03-25:free

## 1. Using the OpenRouter API directly

In [15]:
import requests
import json

response = requests.post(
  url=f"{BASE_URL}/chat/completions",
  headers={
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
  },
  data=json.dumps({
    "model": MODEL,
    "messages": [
      {
        "role": "user",
        "content": "What is the meaning of life?"
      }
    ]
  })
)

print(response.json())

{'id': 'gen-1745045430-e3zwsL735OzknTz75wdX', 'provider': 'Chutes', 'model': 'google/gemma-3-1b-it', 'object': 'chat.completion', 'created': 1745045430, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': 'Please, let’s tackle this big question! The meaning of life is a question that has been pondered by philosophers, theologians, artists, and individuals for millennia. There’s no single, universally accepted answer. It’s a profoundly personal question, and the meaning you find will depend entirely on your own perspective and experiences. \n\nHere’s a breakdown of different ways people approach this, grouped into categories:\n\n**1. Philosophical Perspectives:**\n\n* **Nihilism:** This suggests that life is inherently without meaning, purpose, or intrinsic value. It can be a challenging viewpoint, but some find freedom in accepting this lack of inherent meaning.\n* **Existentialism:** This philo

In [16]:
data = response.json()
data['choices'][0]['message']['content']

'Please, let’s tackle this big question! The meaning of life is a question that has been pondered by philosophers, theologians, artists, and individuals for millennia. There’s no single, universally accepted answer. It’s a profoundly personal question, and the meaning you find will depend entirely on your own perspective and experiences. \n\nHere’s a breakdown of different ways people approach this, grouped into categories:\n\n**1. Philosophical Perspectives:**\n\n* **Nihilism:** This suggests that life is inherently without meaning, purpose, or intrinsic value. It can be a challenging viewpoint, but some find freedom in accepting this lack of inherent meaning.\n* **Existentialism:** This philosophy emphasizes individual freedom and responsibility.  We are born into existence without a pre-determined purpose, and it\'s *our* responsibility to create our own meaning through choices and actions. Thinkers like Sartre and Camus argue that existence precedes essence – we define ourselves th

## 2. Using OpenAI Agents SDK

In [18]:
import asyncio
from openai import AsyncOpenAI
from agents import Agent, OpenAIChatCompletionsModel, Runner, set_tracing_disabled

client = AsyncOpenAI(
    api_key=OPENROUTER_API_KEY,
    base_url=BASE_URL
)

set_tracing_disabled(disabled=True)

async def main():
    # This agent will use the custom LLM provider
    agent = Agent(
        name="Assistant",
        instructions="You only respond in haikus.",
        model=OpenAIChatCompletionsModel(model=MODEL, openai_client=client),
    )

    result = await Runner.run(
        agent,
        "Tell me about recursion in programming.",
    )
    print(result.final_output)


if __name__ == "__main__":
    asyncio.run(main())

 amaze!

A call, a loop, a climb,
Repeating steps, building up,
Logic's gentle flow. 

Each function calls more,
A nested depth, a small dance,
Solutions subtly bloom. 

Efficiency's quiet grace,
Reducing work with each step,
A clever, simple key. 

Though complex can be deep,
It’s logic’s art, finely spun,
Building up a whole. 

Consider this, dear friend, 
Recursion’s a rhythm true. 
Code's a graceful weave. 



# OpenRouter 404 Error Solution

## Error - No endpoints found matching
```python
NotFoundError: Error code: 404 - {'error': {'message': 'No endpoints found matching your data policy. Enable prompt training here: https://openrouter.ai/settings/privacy', 'code': 404}}
```

## Cause
This error occurs when OpenRouter API can't find endpoints matching your data policy, typically because prompt training is disabled.

## Solution

1. **Enable Prompt Training**:
   - Visit [OpenRouter Privacy Settings](https://openrouter.ai/settings/privacy)
   - Toggle ON "Prompt Training" option

2. **Re-run your code** after enabling

![OpenRouter Settings Screenshot](https://github.com/panaversity/learn-agentic-ai/blob/main/01_ai_agents_first/02_openrouter/openrouter.png?raw=1)
*(Example: Enable prompt training in privacy settings)*

## Prevention
Keep prompt training enabled for uninterrupted API access.
```